In [1]:
# Define the project directory inorder to complie with the reproduceability of the code
project_path = """/home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/"""

In [2]:
# Import all the required libraries
import warnings
warnings.filterwarnings('ignore')

import gc
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split


2023-03-31 15:43:16.762393: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-31 15:43:16.792205: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 15:43:17.201745: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv(project_path + "Daphnet-Freezing-of-Gait-Dataset/consolidated_dataset.csv", sep='\t', header=0)
df

,Time in millisecond,Ankle_acc - horizontal forward,Ankle_acc - vertical,Ankle_acc - horizontal lateral,Upper_leg_acc - horizontal forward,Upper_leg_acc - vertical,Upper_leg_acc - horizontal lateral,Trunk_acc - horizontal forward,Trunk_acc - vertical,Trunk_acc - horizontal lateral,Annotation,File S,File R,File
0,15,70,39,-970,0,0,0,0,0,0,0,1.0,1.0,S01R01
1,31,70,39,-970,0,0,0,0,0,0,0,1.0,1.0,S01R01
2,46,60,49,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
3,62,60,49,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
4,78,50,39,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917882,3020296,-131,107,-960,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917883,3020312,-121,127,-970,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917884,3020328,-141,117,-960,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917885,3020343,-131,127,-980,0,0,0,0,0,0,0,10.0,1.0,S10R01


In [4]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return train_test_split(np.array(X), np.array(y), test_size=0.2, shuffle=False)
# X, Y = split_sequences(main_df['S01R01'], 100)

In [5]:
results = dict()
verbose = 1
epochs = 2
batch_size = 128
n_steps = 3
n_features = 9
n_outputs = 3
if "single-output-stacked-LSTM" not in os.listdir(project_path + "models/"):
    os.mkdir(project_path + "models/single-output-stacked-LSTM")
if f"n_steps_{n_steps}" not in os.listdir(project_path + "models/single-output-stacked-LSTM"):
    os.mkdir(project_path + f"models/single-output-stacked-LSTM/n_steps_{n_steps}")

for file_name in df['File'].unique():
    X_train, X_test, y_train, y_test = split_sequences(df[df['File']==file_name].drop(['Time in millisecond', 
                                                           'File S', 
                                                           'File R', 
                                                           'File'], 
                                                          axis=1).values, 
                           n_steps)
    
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    print(file_name)
    hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    print("Accuracy _------------------------------------")
    temp_results = dict()
    temp_results['pred_loss'] , temp_results['pred_accuracy'] = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    model.save(project_path + f"models/single-output-stacked-LSTM/n_steps_{n_steps}/{file_name}.tf")
    for i in range(len(hist.history['loss'])):
        temp_results[f"loss_epoch_{i}"] = hist.history['loss'][i]
    for i in range(len(hist.history['accuracy'])):
        temp_results[f"accuracy_epoch_{i}"] = hist.history['accuracy'][i]    
    results[file_name] = temp_results
    del X_train
    del y_train
    del X_test
    del y_test
    del model
    del temp_results
    del hist
    gc.collect()
file_name = "S00R00"
X_train, X_test, y_train, y_test = split_sequences(df.drop(['Time in millisecond', 
                                                            'File S', 
                                                            'File R', 
                                                            'File'], 
                                                            axis=1).values, 
                                    n_steps)

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
print(file_name)
hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
print("Accuracy _------------------------------------")
temp_results = dict()
temp_results['pred_loss'] , temp_results['pred_accuracy'] = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
model.save(project_path + f"models/single-output-stacked-LSTM/n_steps_{n_steps}/{file_name}.tf")
for i in range(len(hist.history['loss'])):
    temp_results[f"loss_epoch_{i}"] = hist.history['loss'][i]
for i in range(len(hist.history['accuracy'])):
    temp_results[f"accuracy_epoch_{i}"] = hist.history['accuracy'][i]    
results[file_name] = temp_results
del X_train
del y_train
del X_test
del y_test
del model
del temp_results
del hist
gc.collect()


S01R01
Epoch 1/2


2023-03-31 15:43:19.137782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 15:43:19.150875: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


950/950 [==============================] - 7s 6ms/step - loss: 0.5734 - accuracy: 0.2155
Epoch 2/2
950/950 [==============================] - 6s 6ms/step - loss: 0.4413 - accuracy: 0.3194
Accuracy _------------------------------------
238/238 [==============================] - 1s 3ms/step - loss: 0.5655 - accuracy: 0.0935


2023-03-31 15:43:32.892268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,9]
	 [[{{node inputs}}]]
2023-03-31 15:43:32.896848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,9]
	 [[{{node inputs}}]]
2023-03-31 15:43:32.976002: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,100]
	 [[{{node inputs}}]]
2023-03-31 15:

INFO:tensorflow:Assets written to: /home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/models/single-output-stacked-LSTM/n_steps_3/S01R01.tf/assets


...

S00R00
Epoch 1/2
11987/11987 [==============================] - 80s 7ms/step - loss: 0.4729 - accuracy: 0.3486
Epoch 2/2
11987/11987 [==============================] - 82s 7ms/step - loss: 0.4408 - accuracy: 0.3375
Accuracy _------------------------------------
2997/2997 [==============================] - 9s 3ms/step - loss: 0.4579 - accuracy: 0.6220


2023-03-31 15:49:37.328734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,9]
	 [[{{node inputs}}]]
2023-03-31 15:49:37.333185: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,9]
	 [[{{node inputs}}]]
2023-03-31 15:49:37.414179: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,100]
	 [[{{node inputs}}]]
2023-03-31 15:

INFO:tensorflow:Assets written to: /home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/models/single-output-stacked-LSTM/n_steps_3/S00R00.tf/assets


INFO:tensorflow:Assets written to: /home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/models/single-output-stacked-LSTM/n_steps_3/S00R00.tf/assets


58029

In [6]:
pd.DataFrame(results).T

,pred_loss,pred_accuracy,loss_epoch_0,loss_epoch_1,accuracy_epoch_0,accuracy_epoch_1
S01R01,0.565520,0.093463,0.573353,0.441282,0.215498,0.319374
S01R02,0.258300,0.302812,0.554589,0.423588,0.345803,0.345899
S02R01,0.756659,0.219542,0.454851,0.308008,0.173101,0.397126
S02R02,0.271248,0.412070,0.801193,0.664963,0.379452,0.219469
S03R01,0.704475,0.046917,0.659285,0.561944,0.376298,0.369345
S03R02,0.428799,0.315280,0.658843,0.659431,0.337138,0.254635
S03R03,0.398077,0.068450,0.414077,0.339017,0.253464,0.226069
S04R01,0.381077,0.006667,0.465145,0.346345,0.243071,0.356502
S05R01,0.975098,0.463849,0.713044,0.649276,0.327016,0.415007
S05R02,1.016016,0.519182,0.636023,0.557996,0.317551,0.354228


In [7]:
if "single-output-stacked-LSTM" not in os.listdir(project_path + "results"):
    os.mkdir(project_path + f"results/single-output-stacked-LSTM") 
pd.DataFrame(results).T.to_csv(project_path + f"results/single-output-stacked-LSTM/n_steps_{n_steps}.csv", header=True, index=False, sep="\t")